# 1. Import Packages and Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Embedding
import keras.backend as K
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
import xgboost

from transformers import BertTokenizer, TFBertModel

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix

import scipy
import pandas as pd
import numpy as np
import gensim

import nltk
from nltk.data import find
import matplotlib.pyplot as plt
import shap

import matplotlib
import sklearn
import pickle
import random
import multiprocessing

2022-07-17 18:39:31.823600: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print('tf ' + tf.__version__)
print('sklearn ' + sklearn.__version__)
print('xgboost ' + xgboost.__version__)
print('nltk ' + nltk.__version__)
print('pd ' + pd.__version__)
print('np ' + np.__version__)
print('shap ' + shap.__version__)
print('mpl ' + matplotlib.__version__)
print('scipy ' + scipy.__version__)
print('gensim ' + gensim.__version__)

tf 2.9.1
sklearn 0.24.2
xgboost 1.6.1
nltk 3.7
pd 1.3.4
np 1.20.3
shap 0.41.0
mpl 3.4.3
scipy 1.7.1
gensim 4.2.0


# 2. Read in Dataset + Create Train/Test Set

In [3]:
sample_dataset = pd.read_csv('Language_Detection/Train_Test_Data/train.csv')[['Lyric','language label']]
test_dataset = pd.read_csv('Language_Detection/Train_Test_Data/test.csv')[['Lyric','language label']]
print('Label Counts in Train Set')
display(sample_dataset['language label'].value_counts())
train_set = sample_dataset
val_set = test_dataset.iloc[:1418]
test_set = test_dataset.iloc[1418:]

Label Counts in Train Set


Spanish        2412
Portuguese     2405
English        2371
Kinyarwanda    1332
Italian        1156
French          968
German          700
Other           509
Finnish         114
Swedish          97
Romanian         74
Name: language label, dtype: int64

# 3. Resample (Oversample on Minority Classes) Training Set to Deal with Class Imbalance

In [4]:
random.seed(50)
max_class_counts = train_set['language label'].value_counts().iloc[0]
resampled_train_set = pd.DataFrame()
for lang in train_set['language label'].unique():
    subset = train_set[train_set['language label'] == lang].copy()
    if len(subset) == max_class_counts:
        resampled_train_set = pd.concat([resampled_train_set,subset],ignore_index=True)
    else:
        added_subset = subset.iloc[random.choices(np.arange(0,len(subset)),k=max_class_counts - len(subset))]
        resampled_train_set = pd.concat([resampled_train_set,subset,added_subset],ignore_index=True)
        
display(resampled_train_set)

,Lyric,language label
0,Varf&oumlr ska det vara så seri&oumlst f&oumlr...,Swedish
1,Intro:\n(What a group of kids we sent out into...,Swedish
2,"""Vem är Gud? (Vad är Gud?) ""\n""Det är en svår ...",Swedish
3,"vi sover på dagen,\nvi saknar tidsuppfattning,...",Swedish
4,"Honey, honey, underbara, aha, honey honey\nHon...",Swedish
...,...,...
26527,Trece timpul si inteleg ca trece\nDragostea da...,Romanian
26528,Astazi pe la 5 ma vad cu ea\nNu stiu ce m-aste...,Romanian
26529,"can you give me ,can you give me\n\nAstazi pe ...",Romanian
26530,I:\nLasa-ma sa-ti spun :\n'viata mea fara tine...,Romanian


# 4. Term Density Transformation of Text Data

In [5]:
resampled_train_set['language label'].value_counts()

Swedish        2412
French         2412
Kinyarwanda    2412
Spanish        2412
German         2412
Portuguese     2412
Italian        2412
Finnish        2412
English        2412
Other          2412
Romanian       2412
Name: language label, dtype: int64

In [6]:
def preprocess_text(text):
    text = text.lower()
    text = text.replace('\n', ' ')
    text = text.replace('  ',' ')
    return text

vectorizer = CountVectorizer(preprocessor=preprocess_text)

#### Vectorize According to Terms in Non-Other Category

In [7]:
vectorizer.fit(resampled_train_set[resampled_train_set['language label'] != 'Other']['Lyric'])

CountVectorizer(preprocessor=<function preprocess_text at 0x7fb8888b60d0>)

In [8]:
#Vectorize Train Lyrics
train_lyrics = vectorizer.transform(resampled_train_set['Lyric'])
train_lyrics = pd.DataFrame(train_lyrics.todense(),columns = vectorizer.get_feature_names())
train_lyrics_token_count = train_lyrics.sum(axis=1)
train_lyrics = train_lyrics/np.array(train_lyrics_token_count.repeat(len(train_lyrics.columns))).reshape(train_lyrics.shape)

#Vectorize Val Lyrics
val_lyrics = vectorizer.transform(val_set['Lyric'])
val_lyrics = pd.DataFrame(val_lyrics.todense(),columns = vectorizer.get_feature_names(),index=val_set.index)
val_lyrics_token_count = val_lyrics.sum(axis=1)
val_lyrics = val_lyrics/np.array(val_lyrics_token_count.repeat(len(val_lyrics.columns))).reshape(val_lyrics.shape)

#Vectorize Test Lyrics
test_lyrics = vectorizer.transform(test_set['Lyric'])
test_lyrics = pd.DataFrame(test_lyrics.todense(),columns = vectorizer.get_feature_names(),index=test_set.index)
test_lyrics_token_count = test_lyrics.sum(axis=1)
test_lyrics = test_lyrics/np.array(test_lyrics_token_count.repeat(len(test_lyrics.columns))).reshape(test_lyrics.shape)

train_labels = resampled_train_set['language label']
val_labels = val_set['language label']
test_labels = test_set['language label']

#### Convert to float 32 and drop observations that failed to featurize

In [9]:
train_lyrics = train_lyrics.astype('float32')
val_lyrics = val_lyrics.astype('float32')
test_lyrics = test_lyrics.astype('float32')

In [10]:
train_lyrics.dropna(inplace=True)

In [11]:
train_labels = train_labels.loc[train_lyrics.index]

In [12]:
val_lyrics.dropna(inplace=True)

In [13]:
val_labels = val_labels.loc[val_lyrics.index]

In [14]:
test_lyrics.dropna(inplace=True)

In [15]:
test_labels = test_labels.loc[test_lyrics.index]

In [16]:
print(len(train_lyrics))
print(len(val_lyrics))
print(len(test_lyrics))

26484
1415
1616


#### Mapping to map text labels to numeric labels

In [17]:
mapping = {}
count = 0
for label in train_labels.unique():
    mapping[label] = count
    count = count + 1

# 5. Quick Evaluation of Classical ML Models

In [18]:
def optimal_model_id(xtrain,xval,xtest,ytrain,yval,ytest,estimator,param_grid,metric='accuracy'):
    
    #Concatenate training and validation data
    train_val_feats = pd.concat([xtrain,xval],ignore_index=True)
    train_val_labels = pd.concat([ytrain,yval],ignore_index=True)
    #Instantiate Grid Search with model and param grid to ID which hyperparameter combo enables the model to generalize
    #best on the validation set
    grid = GridSearchCV(estimator = estimator, param_grid= param_grid,
                        scoring=metric,cv=[(np.arange(0,len(xtrain)),np.arange(len(xtrain),len(train_val_feats)))])
    
    display(train_val_feats)
    display(train_val_labels.map(mapping))
    grid.fit(train_val_feats,train_val_labels.map(mapping))
    
    #Store Best Performing Model Output
    best_estimator = grid.best_estimator_
    best_val_score = grid.best_score_
    
    #Predictions on test set with optimal model
    test_preds = best_estimator.predict(xtest)
    #performance on test set
    oos_score = accuracy_score(ytest.map(mapping),test_preds)
    label_options = list(ytest.unique())
    
    #Confustion matrix of true for predicted values on the test set
    confuse = pd.DataFrame(confusion_matrix(ytest.map(mapping),test_preds),index = label_options,columns = label_options)
    
    #return optimal model results
    return {'best_estimator':best_estimator,
           'best_val_score':best_val_score,
           'best_test_score':oos_score,
           'metric':metric,
           'test_set_confusion_matrix':confuse}

#### KNN Test

test = optimal_model_id(train_lyrics,val_lyrics,test_lyrics,train_labels,val_labels,test_labels,
                KNeighborsClassifier(),{'n_neighbors':[1,3,5,7,9]},'accuracy')
display(test)
display(test['test_set_confusion_matrix'])

#### XGBoost Classifier Test

test = optimal_model_id(train_lyrics,val_lyrics,test_lyrics,train_labels,val_labels,test_labels,
                XGBClassifier(),{'max_depth':[2,3,4],'max_features':['auto'],'n_estimators':[10]},'accuracy')
display(test)
display(test['test_set_confusion_matrix'])

# 6. Basic Feedforward NN w/ Keras Sequential API and Term Density Representation of Input

#### Input goes sequentially from one hidden layer to the next "left to right"

In [19]:
#Define Model Architecture Sequentially
model = keras.Sequential([
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dense(11,activation='softmax')
])

#Compile the model, specifying loss function, optimizer, and performance metric
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(),
             optimizer = keras.optimizers.Adam(learning_rate=0.01),
             metrics=['accuracy'],
             )

#Fit model and validate on val set between epochs, set multiprocessing
model.fit(x = np.array(train_lyrics),y = train_labels.map(mapping),batch_size=8,epochs=2,
         validation_data=(np.array(val_lyrics),val_labels.map(mapping)),
         use_multiprocessing=True,workers=multiprocessing.cpu_count() - 1)

Epoch 1/2
3311/3311 [==============================] - 116s 35ms/step - loss: 0.1023 - accuracy: 0.9702 - val_loss: 0.1623 - val_accuracy: 0.9795
Epoch 2/2
3311/3311 [==============================] - 115s 35ms/step - loss: 0.0086 - accuracy: 0.9975 - val_loss: 0.2776 - val_accuracy: 0.9781


In [20]:
preds = model.predict(np.array(test_lyrics))

51/51 [==============================] - 1s 10ms/step


In [21]:
accuracy_score(test_labels.map(mapping),[x.argmax() for x in preds])

0.9814356435643564

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               12132000  
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 11)                1111      
                                                                 
Total params: 12,143,211
Trainable params: 12,143,211
Non-trainable params: 0
_________________________________________________________________


# 7. Word Embedding Based Models That Build Vector Representation of Input, Captures General Meaning Before Pass into Feed Forward NN

#### Build Embedding Matrix

In [23]:
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

#construct embedding matrix w/ prebuilt embedding
vocab_dict = model.key_to_index.copy()
embedding_matrix = np.zeros((43982,300))
for word,index in model.key_to_index.items():
    embedding_matrix[index] = model[word]

#Construct custom embedding matrix for this task
vocab_dict_custom = {}
count = 0
for word in vectorizer.get_feature_names():
    vocab_dict_custom[word] = count
    count = count + 1
embedding_matrix_custom = np.random.random((len(vectorizer.get_feature_names()) + 1,300))
embedding_matrix_custom[-1] = 0

#### Map tokens in train, val, test set to row in embedding matrices for both word2vec and custom embedding matrix

In [24]:
def text_to_index(text_data,mapping,max_size):
    return_data = []
    for text in text_data:
        new_text = text.lower()
        new_text = text.replace('\n',' ')
        new_text = text.replace('  ',' ')
        new_text = new_text.split()
        mapped_text = []
        for token in new_text:
            try:
                mapped_text.append(mapping[token])
            except:
                mapped_text.append(len(mapping))
        
        if len(mapped_text) > max_size:
            mapped_text = mapped_text[:max_size]
        else:
            while len(mapped_text) < max_size:
                mapped_text.append(len(mapping))
                
        return_data.append(mapped_text)
    
    return return_data

In [25]:
train_tokens_prebuilt = text_to_index(resampled_train_set['Lyric'].loc[train_lyrics.index],vocab_dict,1000)
train_tokens_custom = text_to_index(resampled_train_set['Lyric'].loc[train_lyrics.index],vocab_dict_custom,1000)

val_tokens_prebuilt = text_to_index(val_set['Lyric'].loc[val_lyrics.index],vocab_dict,1000)
val_tokens_custom = text_to_index(val_set['Lyric'].loc[val_lyrics.index],vocab_dict_custom,1000)

test_tokens_prebuilt = text_to_index(test_set['Lyric'].loc[test_lyrics.index],vocab_dict,1000)
test_tokens_custom = text_to_index(test_set['Lyric'].loc[test_lyrics.index],vocab_dict_custom,1000)

### Deep Averaging Network (DAN) w/ Functional Keras API and Custom Embedding Matrix

In [26]:
def create_dan_model(retrain_embeddings=False, 
                     max_sequence_length=1000,
                     embedding_matrix=embedding_matrix_custom, 
                     hidden_dim=[100,100,100],
                     dropout_rate=0.3,
                     hidden_layer_activation = 'relu',
                     output_layer_size = 4,
                     output_activation = 'softmax',
                     learning_rate=0.001):
    """
    Construct the DAN model including the compilation and return it. Parametrize it using the arguments.
    retrain_embeddings: bool, indicates whether embeddings are retrainable
    max_sequence_length: Number of token IDs to expect in a given input
    embedding_matrix: initialize embedding layer with embedding matrix, specifying weights
    hidden_dim = number of neurons in hidden layers
    dropout = dropout rate
    output_layer_size = # of neurons in output layer corresponding to # of classes, each neuron predicts P(class K | x)
    output_activation = activation function for output layer
    learning_rate = learning rate for gradient descent for finding model params to optimize loss
    """
    
    #Specify Embedding Layer, including shape, intialize with weights, expected input length, and whether it is trainable
    dan_embedding_layer = Embedding(embedding_matrix.shape[0],
                                  embedding_matrix.shape[1],
                                  weights = [embedding_matrix],
                                  input_length=max_sequence_length,
                                  trainable=retrain_embeddings,
                                   name = 'embedding_layer')
    
    
    #Input Layer, sequence of max_sequence_length tokens
    dan_input_layer = tf.keras.layers.Input(shape=(max_sequence_length,), dtype='int64',name='input')
    #Inputs go into embedding layer, form max_sequence_length x embedding dim matrix
    dan_embeddings = dan_embedding_layer(dan_input_layer)
    #Embeddings are averaged, forming single vector represenation of size embedding matrix
    dan_avg_input_embeddings = tf.keras.layers.Lambda(lambda x: K.mean(x, axis=1), name='averaging')(dan_embeddings)
    
    #input into hidden layers
    x = dan_avg_input_embeddings #hidden layer initial input
    count = 1
    for layer in hidden_dim:
        hidden = tf.keras.layers.Dense(layer,activation = hidden_layer_activation,name='hidden_' + str(count))(x)
        dropout = tf.keras.layers.Dropout(dropout_rate,name='dropout_' + str(count))(hidden)
        count = count + 1
        x = dropout
        
    dan_classification = tf.keras.layers.Dense(output_layer_size, activation=output_activation, name='classification')(x)
    dan_model = tf.keras.models.Model(inputs=dan_input_layer, outputs=[dan_classification])
    dan_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate,
                                                beta_1=0.9,
                                                beta_2=0.999,
                                                epsilon=1e-07,
                                                amsgrad=False,
                                                name='Adam'),
                 metrics='accuracy')
    
    print(dan_model.summary())

    return dan_model

In [40]:
dan_model_sorted = create_dan_model(retrain_embeddings=True,embedding_matrix=embedding_matrix_custom,
                                   output_layer_size=11)
# dan_sorted_history = dan_model_sorted.fit(np.array(train_tokens_custom),
#                         np.array(train_labels.map(mapping)),
#                         validation_data=(np.array(val_tokens_custom), np.array(val_labels.map(mapping))),
#                         batch_size=8,
#                         epochs=2,
#                         shuffle=True,
#                         use_multiprocessing=True,workers=multiprocessing.cpu_count() - 1)

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 1000)]            0         
                                                                 
 embedding_layer (Embedding)  (None, 1000, 300)        36396000  
                                                                 
 averaging (Lambda)          (None, 300)               0         
                                                                 
 hidden_1 (Dense)            (None, 100)               30100     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 hidden_2 (Dense)            (None, 100)               10100     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0   

In [ ]:
accuracy_score(test_labels.map(mapping),[x.argmax() for x in dan_model_sorted.predict(test_tokens_custom)])

In [ ]:
dan_model_sorted.weights[0].shape

### Weighted Attention Network (WAN) with Custom Embeddings, allows for computation of multiple attention based representations of input before a final attention layer learns how to balance attention vectors from prior layer

In [28]:
def create_wan_model(retrain_embeddings=False, 
                     max_sequence_length=1000,
                     embedding_matrix=embedding_matrix,
                     num_attention = 1,
                     hidden_dim=[100,100,100],
                     dropout_rate=0.3,
                     hidden_layer_activation = 'relu',
                     output_layer_size = 4,
                     output_activation = 'softmax',
                     learning_rate=0.001):
    """
    Construct the WAN model including the compilation and return it. Parametrize it using the arguments.
    retrain_embeddings: bool, indicates whether embeddings are retrainable
    max_sequence_length: Number of token IDs to expect in a given input
    embedding_matrix: initialize embedding layer with embedding matrix, specifying weights
    num_attention = number of parallel attention computations that learn how to balance embeddings into a single
    vector representation, final attention layer weights prior attention based representations
    hidden_dim = number of neurons in hidden layers
    dropout = dropout rate
    output_layer_size = # of neurons in output layer corresponding to # of classes, each neuron predicts P(class K | x)
    output_activation = activation function for output layer
    learning_rate = learning rate for gradient descent for finding model params to optimize loss
    """
    
    #Specify Embedding Layer, including shape, intialize with weights, expected input length, and whether it is trainable
    wan_embedding_layer = Embedding(embedding_matrix.shape[0],
                                  embedding_matrix.shape[1],
                                  weights = [embedding_matrix],
                                  input_length=max_sequence_length,
                                  trainable=retrain_embeddings,
                                   name = 'embedding_layer')
    
    
    #Input Layer, sequence of max_sequence_length tokens
    wan_input_layer = tf.keras.layers.Input(shape=(max_sequence_length,), dtype='int64',name='input')
    #Inputs go into embedding layer, form max_sequence_length x embedding dim matrix
    wan_embeddings = wan_embedding_layer(wan_input_layer)
    
    if num_attention > 0: #If attention is applied to embeddings to learn how to weight into single representation
        if num_attention > 1:
            #Create attention based single vector representations of words according to alternative query vectors
            attention_embeddings = []
            for num in range(num_attention):
                #Apply Query Vector to words in embeddings, returning a max_sequence_length x 1 tensor
                l1_query = tf.keras.layers.Dense(1,activation='linear',use_bias=False,name='attention_query' + str(num+1))(wan_embeddings)
                #reshape to 1 x max_sequence_length
                l1_reshape_query = tf.keras.layers.Reshape((1,max_sequence_length))(l1_query)
                #Softmax over query * key (words) to obtain weights
                l1_weights = tf.keras.layers.Lambda(lambda x:tf.keras.activations.softmax(x),
                                                    name='attention_weights' + str(num+1))(l1_reshape_query)
                #weight embeddings according to weights
                l1_attention = tf.keras.layers.Flatten()(tf.keras.layers.Dot((1,2))((wan_embeddings,l1_weights)))
                attention_embeddings.append(l1_attention)

            concat_attention = tf.keras.layers.Concatenate()(attention_embeddings)
            concat_attention = tf.keras.layers.Reshape((num_attention,embedding_matrix.shape[1]))(concat_attention)
        else:
            concat_attention = wan_embeddings
            num_attention = max_sequence_length
    
        #Apply Query Vector to attention based representations, returning a num_attention x 1 tensor
        wan_query = tf.keras.layers.Dense(1,activation='linear',use_bias=False,name='attention_query')(concat_attention)
        #reshape to 1 x num_attention
        reshaped_query = tf.keras.layers.Reshape((1,num_attention))(wan_query)
        #Softmax over query * key (words) to obtain weights
        wan_weights = tf.keras.layers.Lambda(lambda x:tf.keras.activations.softmax(x),
                                            name='attention_weights')(reshaped_query)
        #weight attention embeddings according to weights, learning how to balance attention based vector representations 
        #from prior layer
        embedding = tf.keras.layers.Flatten()(tf.keras.layers.Dot((1,2))((concat_attention,wan_weights)))
    else: #Default to DAN Treatment of Embeddings if num_attention = 0
        embedding = tf.keras.layers.Lambda(lambda x: K.mean(x, axis=1), name='averaging')(wan_embeddings)
    
    #input into hidden layers
    x = embedding #hidden layer initial input
    count = 1
    for layer in hidden_dim:
        hidden = tf.keras.layers.Dense(layer,activation = hidden_layer_activation,name='hidden_' + str(count))(x)
        dropout = tf.keras.layers.Dropout(dropout_rate,name='dropout_' + str(count))(hidden)
        count = count + 1
        x = dropout
        
    wan_classification = tf.keras.layers.Dense(output_layer_size, activation=output_activation, name='classification')(x)
    wan_model = tf.keras.models.Model(inputs=wan_input_layer, outputs=[wan_classification])
    wan_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate,
                                                beta_1=0.9,
                                                beta_2=0.999,
                                                epsilon=1e-07,
                                                amsgrad=False,
                                                name='Adam'),
                 metrics=['accuracy'],
                     run_eagerly=True)
    
    print(wan_model.summary())

    return wan_model


In [41]:
wan_model_sorted = create_wan_model(retrain_embeddings=False,embedding_matrix=embedding_matrix_custom,
                                   num_attention=2,output_layer_size=11)
# wan_sorted_history = wan_model_sorted.fit(np.array(train_tokens_custom),
#                         np.array(train_labels.map(mapping)),
#                         validation_data=(np.array(val_tokens_custom), np.array(val_labels.map(mapping))),
#                         batch_size=8,
#                         epochs=2,
#                         shuffle=True)

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 1000)]       0           []                               
                                                                                                  
 embedding_layer (Embedding)    (None, 1000, 300)    36396000    ['input[0][0]']                  
                                                                                                  
 attention_query1 (Dense)       (None, 1000, 1)      300         ['embedding_layer[0][0]']        
                                                                                                  
 attention_query2 (Dense)       (None, 1000, 1)      300         ['embedding_layer[0][0]']        
                                                                                           

In [ ]:
accuracy_score(test_labels.map(mapping),[x.argmax() for x in wan_model_sorted.predict(test_tokens_custom)])

# 8. BERT Based Models to Develop Contextual Representations

#### Pretrained Tokenizer

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#### Tokenize Text and Extract Input IDs For Each Token in Each Example

In [ ]:
train_bert_ids = bert_tokenizer(list(train_set['Lyric']),
                               max_length=512,truncation=True,padding='max_length', return_tensors='tf')['input_ids']
val_bert_ids = bert_tokenizer(list(val_set['Lyric']),
                             max_length=512,truncation=True,padding='max_length', return_tensors='tf')['input_ids']
test_bert_ids = bert_tokenizer(list(test_set['Lyric']),
                              max_length=512,truncation=True,padding='max_length', return_tensors='tf')['input_ids']

#### BERT Model to Generate Contextual Embeddings

In [ ]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

#### CLS Token Example

In [ ]:
bert_model(train_bert_ids[:1])[0][:,0]

#### Pooled Token Example

In [ ]:
bert_model(train_bert_ids[:1])[1].shape[0]

In [ ]:
def create_bert_model(train_layers=-1,
                      embedding_dim=768,
                      token = 'cls', # 'cls' or 'pooled' or 'avg'
                      num_attention = 0,
                      hidden_dim=[100,100,100],
                      dropout_rate=0.3,
                      hidden_layer_activation = 'relu',
                      output_layer_size = 4,
                      output_activation = 'softmax',
                      learning_rate=0.001):
    """
    Build a simple classification model with BERT.
    """
    
    # Load BERT
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')

    #restrict training to the train_layers outer transformer layers (SPECIFY WHICH BERT LAYERS ARE TRAINABLE)
    if not train_layers == -1:

            retrain_layers = []

            for retrain_layer_number in range(train_layers):

                layer_code = '_' + str(11 - retrain_layer_number)
                retrain_layers.append(layer_code)

            for w in bert_model.weights:
                if not any([x in w.name for x in retrain_layers]):
                    w._trainable = False
    
    #Input Layer
    input_ids = tf.keras.layers.Input(shape = (512,),dtype=tf.int64, name='input_ids_layer') 
    #Get Contextual Embeddings + Single Vector Representations of Input (CLS or Pooled)
    bert_out = bert_model(input_ids) 
    
    if token == 'cls':
        token = bert_out[0][:,0] #Get CLS Tokens
    elif token == 'pooled':
        token = bert_out[1] #Pooled Token
    elif token == 'avg':
        token = tf.math.reduce_mean(bert_out[0][:,1:-1],axis=1)
    elif token == 'word_embeddings':
        token = bert_out[0][:,1:-1]
    
    # Attention to Combine CLS/Pooled Tokens into single representation in the event of chunking text for single example
    if num_attention == 0: # Single CLS/Pooled Token
        embedding = token
    elif num_attention == 1:
        #Apply Query Vector to BERT Token, returning a num_attention x 1 tensor
        query = tf.keras.layers.Dense(1,activation='linear',use_bias=False,name='attention_query')(token)
        if token.shape == (None,768):
            reshaped_query = tf.keras.layers.Reshape((1,1))(query)
            token = tf.keras.layers.Reshape((1,token.shape[1]))(token)
        else:
            reshaped_query = tf.keras.layers.Reshape((1,token.shape[1]))(query)
        #Softmax over query * key (words) to obtain weights
        weights = tf.keras.layers.Lambda(lambda x:tf.keras.activations.softmax(x),
                                            name='attention_weights')(reshaped_query)
        #weight attention embeddings according to weights
        embedding = tf.keras.layers.Flatten()(tf.keras.layers.Dot((1,2))((token,weights)))
    else:
        #Create attention based single vector representations of words according to alternative query vectors
        attention_embeddings = []
        for num in range(num_attention):
            #Apply Query Vector to words in embeddings, returning a embedding_dim x 1 tensor
            l1_query = tf.keras.layers.Dense(1,activation='linear',use_bias=False,name='attention_query_l' + str(num+1))(token)
            if token.shape == (None,768):
                l1_reshaped_query = tf.keras.layers.Reshape((1,1))(l1_query)
                l1_token = tf.keras.layers.Reshape((1,token.shape[1]))(token)
            else:
                l1_reshaped_query = tf.keras.layers.Reshape((1,token.shape[1]))(l1_query)
                l1_token = token
                
            #Softmax over query * key (words) to obtain weights
            l1_weights = tf.keras.layers.Lambda(lambda x:tf.keras.activations.softmax(x),
                                                name='attention_weights_l' + str(num+1))(l1_reshaped_query)
            
            #weight attention embeddings according to weights
            l1_attention = tf.keras.layers.Flatten()(tf.keras.layers.Dot((1,2))((l1_token,l1_weights)))
            attention_embeddings.append(l1_attention)

        concat_attention = tf.keras.layers.Concatenate()(attention_embeddings)
        concat_attention = tf.keras.layers.Reshape((num_attention,embedding_dim))(concat_attention)
        
        #Apply Query Vector to BERT Embeddings with Various Attention-Based representations, returning a num_attention x 1 tensor
        query = tf.keras.layers.Dense(1,activation='linear',use_bias=False,name='attention_query')(concat_attention)
        #reshape to 1 x num_attention
        reshaped_query = tf.keras.layers.Reshape((1,num_attention))(query)
        #Softmax over query * key (words) to obtain weights
        weights = tf.keras.layers.Lambda(lambda x:tf.keras.activations.softmax(x),
                                            name='attention_weights')(reshaped_query)
        #weight attention embeddings according to weights
        embedding = tf.keras.layers.Flatten()(tf.keras.layers.Dot((1,2))((concat_attention,weights)))
        
    x = embedding
    count = 1
    for layer in hidden_dim:
        hidden = tf.keras.layers.Dense(layer,activation = hidden_layer_activation,name='hidden_' + str(count))(x)
        dropout = tf.keras.layers.Dropout(dropout_rate,name='dropout_' + str(count))(hidden)
        count = count + 1
        x = dropout

    bert_classification = tf.keras.layers.Dense(output_layer_size, activation=output_activation,name='classification_layer')(x)
    
    bert_model = tf.keras.Model(inputs=[input_ids], outputs=[bert_classification])
    
    bert_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate,
                                                beta_1=0.9,
                                                beta_2=0.999,
                                                epsilon=1e-07,
                                                amsgrad=False,
                                                name='Adam'),
                 metrics=['accuracy'],
                     run_eagerly=True) 
    
    print(bert_model.summary())
    
    return bert_model


In [ ]:
cls_bert_model = create_bert_model(learning_rate=0.0005,output_layer_size=11,num_attention=5,token='word_embeddings')
                        
cls_bert_model.fit(train_bert_ids[:100], train_set['language label'].map(mapping).iloc[:100], 
                   validation_data=(val_bert_ids[:100],val_set['language label'].map(mapping).iloc[:100]),
                   batch_size=8, epochs=2)  

# 9. CNN Based Embeddings that Develop Feature Map Based Representations of Embeddings that Retain Word Order

In [29]:
def create_cnn_model(num_filters = [100, 100, 50, 25],
                     kernel_sizes = [3, 5, 10, 20], 
                     retrain_embeddings=False, 
                     max_sequence_length=1000,
                     embedding_matrix=embedding_matrix,
                     hidden_dim=[100,100,100],
                     dropout_rate=0.3,
                     hidden_layer_activation = 'relu',
                     output_layer_size = 4,
                     output_activation = 'softmax',
                     learning_rate=0.001):
    """
    Build a  classification model that applies Conv layers to word embeddings to generate feature maps of word embeddings
    in order to construct a order conscious, meaning based representation
    """

    ### YOUR CODE HERE
    
    #Specify Embedding Layer, including shape, intialize with weights, expected input length, and whether it is trainable
    embedding_layer = Embedding(embedding_matrix.shape[0],
                                  embedding_matrix.shape[1],
                                  weights = [embedding_matrix],
                                  input_length=max_sequence_length,
                                  trainable=retrain_embeddings,
                                   name = 'embedding_layer')
    
    #Input Layer
    input_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='input_ids_layer') 
    
    #Get Word Embeddings for Input
    word_embeddings = embedding_layer(input_ids)
    
    conv_layers_for_all_kernel_sizes = []
    for kernel_size, filters in zip(kernel_sizes, num_filters):
        conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(word_embeddings)
        conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
        conv_layers_for_all_kernel_sizes.append(conv_layer)
    
    h = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1)
    h = keras.layers.Dropout(rate=dropout)(h)
    
    x = h
    count = 1
    for layer in hidden_dim:
        hidden = tf.keras.layers.Dense(layer,activation = hidden_layer_activation,name='hidden_' + str(count))(x)
        dropout = tf.keras.layers.Dropout(dropout_rate,name='dropout_' + str(count))(hidden)
        count = count + 1
        x = dropout
        
    classification = tf.keras.layers.Dense(output_layer_size, activation=output_activation,name='classification_layer')(x)
    classification_model = tf.keras.Model(inputs=[input_ids], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                            loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                            metrics='accuracy') 
    
    print(classification_model.summary())
    ### END YOUR CODE
    
    return classification_model

In [30]:
cnn_model = create_wan_model(retrain_embeddings=False,embedding_matrix=embedding_matrix_custom,
                             output_layer_size=11)

cnn_model_history = cnn_model.fit(np.array(train_tokens_custom),
                        np.array(train_labels.map(mapping)),
                        validation_data=(np.array(val_tokens_custom), np.array(val_labels.map(mapping))),
                        batch_size=8,
                        epochs=2,
                        shuffle=True)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 1000)]       0           []                               
                                                                                                  
 embedding_layer (Embedding)    (None, 1000, 300)    36396000    ['input[0][0]']                  
                                                                                                  
 attention_query (Dense)        (None, 1000, 1)      300         ['embedding_layer[0][0]']        
                                                                                                  
 reshape (Reshape)              (None, 1, 1000)      0           ['attention_query[0][0]']        
                                                                                            

# 10. Chaining Model's Together - Applicable for Multi-Input / Multi-Output Models

In [48]:
# -------------------------------Model that accepts input and creates embedding matrix-------------------------------
input_layer = tf.keras.layers.Input(shape=(1000,))
#Specify Embedding Layer, including shape, intialize with weights, expected input length, and whether it is trainable
embedding_layer = Embedding(embedding_matrix_custom.shape[0],
                            embedding_matrix_custom.shape[1],
                            weights = [embedding_matrix_custom],
                            input_length=1000,
                            trainable=True,
                            name = 'embedding_layer')
embeddings = embedding_layer(input_layer)
embedding_model = tf.keras.Model(inputs = [input_layer],outputs=[embeddings])

# -----------------------------------------------------DAN MODEL-----------------------------------------------------
avg_embedding = tf.keras.layers.Lambda(lambda x:K.mean(x,axis=1))(embedding_model.output)
avg_embedding = tf.keras.Model(inputs = [embedding_model.input], outputs = [avg_embedding])

# -----------------------------------------------------WAN Model-----------------------------------------------------
#Apply Query Vector to attention based representations, returning a num_attention x 1 tensor
query = tf.keras.layers.Dense(1,activation='linear',use_bias=False,name='attention_query')(embedding_model.output)
#reshape to 1 x num_attention
reshaped_query = tf.keras.layers.Reshape((1,1000))(query)
#Softmax over query * key (words) to obtain weights
weights = tf.keras.layers.Lambda(lambda x:tf.keras.activations.softmax(x),
                                    name='attention_weights')(reshaped_query)
#weight attention embeddings according to weights, learning how to balance attention based vector representations 
#from prior layer
wan_embedding = tf.keras.layers.Flatten()(tf.keras.layers.Dot((1,2))((embedding_model.output,weights)))
wan_embedding = tf.keras.Model(inputs=[embedding_model.input],outputs=[wan_embedding])

# WAN Model that uses an attention layer with a single node to learn how to combine WAN/DAN embeddings into single representation
dual_embedding = tf.keras.layers.concatenate([avg_embedding.output,wan_embedding.output])
dual_embedding = tf.keras.layers.Reshape((2,embedding_matrix_custom.shape[1]))(dual_embedding)
query = tf.keras.layers.Dense(1,activation='linear',use_bias=False)(dual_embedding)
reshaped_query = tf.keras.layers.Reshape((1,2))(query)
weights = tf.keras.layers.Lambda(lambda x:tf.keras.activations.softmax(x))(reshaped_query)
embedding = tf.keras.layers.Flatten()(tf.keras.layers.Dot((1,2))((dual_embedding,weights)))
hidden = tf.keras.layers.Dense(100,activation='relu')(embedding)
output = tf.keras.layers.Dense(11,activation='softmax')(hidden)
final_model = tf.keras.Model(inputs=[embedding_model.input],outputs=[output])

final_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                            loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                            metrics='accuracy') 

In [49]:
final_model_history = final_model.fit(np.array(train_tokens_custom),
                        np.array(train_labels.map(mapping)),
                        validation_data=(np.array(val_tokens_custom), np.array(val_labels.map(mapping))),
                        batch_size=8,
                        epochs=2,
                        shuffle=True)

Epoch 1/2
3311/3311 [==============================] - 817s 247ms/step - loss: 0.3181 - accuracy: 0.9071 - val_loss: 0.1337 - val_accuracy: 0.9689
Epoch 2/2
3311/3311 [==============================] - 872s 264ms/step - loss: 0.0568 - accuracy: 0.9844 - val_loss: 0.1450 - val_accuracy: 0.9654


### Example Skeleton Code for Multiinput Model from https://pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/
inputA = Input(shape=(32,))  
inputB = Input(shape=(128,))
#### the first branch operates on the first input
x = Dense(8, activation="relu")(inputA)  
x = Dense(4, activation="relu")(x)  
x = Model(inputs=inputA, outputs=x)
#### the second branch opreates on the second input
y = Dense(64, activation="relu")(inputB)  
y = Dense(32, activation="relu")(y)  
y = Dense(4, activation="relu")(y)  
y = Model(inputs=inputB, outputs=y)
#### combine the output of the two branches
combined = concatenate([x.output, y.output])
#### apply a FC layer and then a regression prediction on the combined outputs
z = Dense(2, activation="relu")(combined)  
z = Dense(1, activation="linear")(z)
#### our model will accept the inputs of the two branches and then output a single value
model = Model(inputs=[x.input, y.input], outputs=z)